# Análise Comparativa

In [126]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn import model_selection
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# 1. Obtenção dos dados


Nessa fase do processo obtemos o conjunto de dados e dicionário de dados.

In [65]:
df = pd.read_csv('../data/raw/obesity_dataset.csv')
df_dict = pd.read_csv('../data/external/dictionary.csv')

df_dict

,variavel,descricao,tipo,subtipo
0,Age,idade do indivíduo,quantitativa,contínua
1,Gender,gênero do indivíduo,qualitativa,nominal
2,Height,altura do indiíduo,quantitativa,contínua
3,Weight,peso do indivíduo,quantitativa,contínua
4,CALC,frequência do consumo de álcool pelo indivíduo,qualitativa,ordinal
5,FAVC,indica se o indivíduo consome comidas altament...,qualitativa,nominal
6,FCVC,indica o nível de consumo de vegetais nas refe...,quantitativa,discreta
7,NCP,quantas refeições principais o indivíduo faz d...,quantitativa,contínua
8,SCC,indica se o indivíduo monitora as calorias ing...,qualitativa,nominal
9,SMOKE,indica se o indivíduo fuma ou não,qualitativa,nominal


# 2. Preparação dos dados

Nessa fase realizamos as principais transformações no conjunto de dados bruto já visando a modelagem.

Para as variáveis nominais, utilizamos a técnica do [OneHotEncoder](http://https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) para fazer o encoding das variáveis categóricas. Para as variáveis ordinais usamos o [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html), e empregamos o [StandardScaler](http://https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#standardscaler) para normalização das variáveis contínuas, ordinais e discretas.

Não aplicamos nenhum método para tratar dados ausentes, pois o conjunto de dados não possui valores faltantes.

In [70]:
target_column = 'NObeyesdad'
colunas_nominal = (
    df_dict
    .query("subtipo == 'nominal'") # Filtrando colunas nominais e tirando a variavel alvo.
    .variavel
    .to_list()
)
colunas_continua = (
    df_dict
    .query("subtipo == 'contínua'") # Filtrando colunas contínuas.
    .variavel
    .to_list()
)
colunas_ordinal = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != @target_column") # Filtrando colunas ordinais e tirando a variavel alvo.
    .variavel
    .to_list()
)
colunas_discreta = (
    df_dict
    .query("subtipo == 'discreta'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1) # Dropando a variável alvo para ficarmos somente com as variáveis independentes.
y = df[target_column]

In [67]:
preprocessador_nominal = Pipeline([ 
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), 
])
preprocessador_continua = Pipeline([ 
    ('normalization', StandardScaler())
])
preprocessador_ordinal = Pipeline([
    ('encoding', OrdinalEncoder()),
    ('normalization', StandardScaler())
    
])
preprocessador_discreta = Pipeline([
    ('normalization', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('nominal', preprocessador_nominal, colunas_nominal),
    ('continuous', preprocessador_continua, colunas_continua),
    ('ordinal', preprocessador_ordinal, colunas_ordinal),
    ('discreta', preprocessador_discreta, colunas_discreta)
])


# 3. Metodologia

Iremos análisar quatro modelos, que serão testados utilizando o KFold como método de validação, a saber:

- Logistic Regression
- K-Nearest-Neighbors
- Support Vector Machine
- Decision Tree

Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Utilizaremos as seguintes métricas para análise:

- **Acurácia (accuracy)**: proporção entre os dados que foram corretamente previstos (como positivos ou negativos) com o total de dados observados;
- **Precisão (precision)**: proporção entre dados corretamente previstos como positivos e o total de observações positivas.
- **Recall**: proporção entre dados corretamente previstos como positivos com o total de observações.
- **Matriz de Confusão**: A matriz de confusão fornece uma visão detalhada das previsões do modelo em comparação com os valores reais.

# 4. Configuração do experimento

In [147]:
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = 0.2
random_state = 42
scoring = 'accuracy'
metrics = ['accuracy', 'precision_macro', 'recall_macro'] 

max_iter = 1000
models = [
    ('K-Nearest Neighbors', KNeighborsClassifier(), {"n_neighbors": range(3, 20, 2), 'weights': ['uniform', 'distance'], 'metrics':['minkowski', 'euclidean', 'manhattan'], 'p':[1,2]}),
    ('Decision Tree', DecisionTreeClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8]}),
    ('Logistic Regression', LogisticRegression(), {'C' : np.logspace(-4, 4, 20)}),

]

**Validação e treinamento com o modelo de Logistic Regression**

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression(max_iter=10000)

pipeline = make_pipeline(StandardScaler(), model)

pipeline.fit(X_train, y_train)

y_test_pred = pipeline.predict(X_test)

print('Avaliação do Modelo Final:')
print(f'  Acurácia: {accuracy_score(y_test, y_test_pred):.4f}')
print(f'  Precisão: {precision_score(y_test, y_test_pred, average="weighted"):.4f}')
print(f'  Revocação: {recall_score(y_test, y_test_pred, average="weighted"):.4f}')
print(f'  Matriz de Confusão:\n{confusion_matrix(y_test, y_test_pred)}')
print(f'  Relatório de Classificação:\n{classification_report(y_test, y_test_pred)}')

Avaliação do Modelo Final:
  Acurácia: 0.8628
  Precisão: 0.8641
  Revocação: 0.8628
  Matriz de Confusão:
[[84  2  0  0  0  0  0]
 [11 63  0  0  0 14  5]
 [ 0  0 90  7  2  0  3]
 [ 0  0  2 86  0  0  0]
 [ 0  0  0  1 97  0  0]
 [ 0  9  0  0  0 63 16]
 [ 0  0  4  0  0 11 64]]
  Relatório de Classificação:
                     precision    recall  f1-score   support

Insufficient_Weight       0.88      0.98      0.93        86
      Normal_Weight       0.85      0.68      0.75        93
     Obesity_Type_I       0.94      0.88      0.91       102
    Obesity_Type_II       0.91      0.98      0.95        88
   Obesity_Type_III       0.98      0.99      0.98        98
 Overweight_Level_I       0.72      0.72      0.72        88
Overweight_Level_II       0.73      0.81      0.77        79

           accuracy                           0.86       634
          macro avg       0.86      0.86      0.86       634
       weighted avg       0.86      0.86      0.86       634



 - Desempenho Geral: O modelo de Regressão Logística apresentou um desempenho satisfatório com boas métricas de acurácia, precisão e revocação, tanto na validação cruzada quanto no conjunto de teste. A acurácia média durante a validação cruzada indica que o modelo tem uma boa capacidade de generalização sobre diferentes subconjuntos dos dados. A precisão e a revocação médias sugerem que o modelo está bem ajustado para identificar tanto as classes positivas quanto negativas.

- Precisão: A precisão alta sugere que o modelo tem um bom desempenho na identificação correta das classes positivas. Se houver um desequilíbrio entre as classes, pode ser necessário ajustar o modelo para melhorar a precisão.

- Revocação: A revocação alta indica que o modelo é eficaz na recuperação de todas as instâncias relevantes. Se a revocação estiver baixa, pode ser necessário ajustar o modelo para reduzir falsos negativos.

- Matriz de Confusão: A matriz de confusão fornece uma visão detalhada das previsões do modelo em comparação com os valores reais. A análise da matriz de confusão pode ajudar a identificar padrões de erros, como a quantidade de falsos positivos e falsos negativos, e pode guiar ajustes futuros no modelo.

**Pontos Fortes e Limitações:**

- Pontos Fortes: O modelo de Regressão Logística é simples, eficiente e relativamente fácil de interpretar. Seus resultados mostram que é uma boa escolha inicial para o problema de classificação abordado.

- Limitações: Apesar do desempenho geral positivo, a Regressão Logística pode ter limitações em termos de modelagem de relações não lineares complexas. Se o desempenho não atender às expectativas, pode ser interessante explorar modelos mais complexos, como máquinas de vetores de suporte (SVM), árvores de decisão, ou redes neurais.
---


**Ajuste de Hiperparâmetros para o Modelos LogisticRegression**

In [145]:
print('Melhores parâmetros encontrados:')
print(grid_search.best_params_)


best_model = grid_search.best_estimator_


y_test_pred = best_model.predict(X_test)


print('\nAvaliação do Modelo Final:')
print(f'  Acurácia: {accuracy_score(y_test, y_test_pred):.4f}')
print(f'  Precisão: {precision_score(y_test, y_test_pred, average="weighted"):.4f}')
print(f'  Revocação: {recall_score(y_test, y_test_pred, average="weighted"):.4f}')
print(f'  Matriz de Confusão:\n{confusion_matrix(y_test, y_test_pred)}')
print(f'  Relatório de Classificação:\n{classification_report(y_test, y_test_pred)}')

Melhores parâmetros encontrados:
{'logisticregression__C': 100, 'logisticregression__penalty': 'l1', 'logisticregression__solver': 'saga'}

Avaliação do Modelo Final:
  Acurácia: 0.9495
  Precisão: 0.9504
  Revocação: 0.9495
  Matriz de Confusão:
[[85  1  0  0  0  0  0]
 [ 8 80  0  0  0  5  0]
 [ 0  0 98  3  0  0  1]
 [ 0  0  1 87  0  0  0]
 [ 0  0  0  1 97  0  0]
 [ 0  3  0  0  0 83  2]
 [ 0  0  3  0  0  4 72]]
  Relatório de Classificação:
                     precision    recall  f1-score   support

Insufficient_Weight       0.91      0.99      0.95        86
      Normal_Weight       0.95      0.86      0.90        93
     Obesity_Type_I       0.96      0.96      0.96       102
    Obesity_Type_II       0.96      0.99      0.97        88
   Obesity_Type_III       1.00      0.99      0.99        98
 Overweight_Level_I       0.90      0.94      0.92        88
Overweight_Level_II       0.96      0.91      0.94        79

           accuracy                           0.95       634
   

- O modelo de Regressão Logística ajustado com GridSearchCV apresentou um desempenho sólido com acurácia e métricas de precisão e revocação equilibradas. A análise detalhada e o ajuste de hiperparâmetros foram eficazes para melhorar o desempenho do modelo. No entanto, sempre há espaço para melhorias, e explorar outros modelos ou técnicas pode fornecer insights adicionais e oportunidades para otimizar ainda mais o desempenho.
---

# 5. Resultados e discussão